<a href="https://colab.research.google.com/github/djy00/ckad5_2nd/blob/main/YOLOv5_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터셋 관리와 라벨링은 roboflow를 이용하였습니다.
# 데이터셋 export는 파일과 다운로드 코드 형식으로 가능합니다.
# 해당 코드는 다운로드 코드를 사용예시 입니다.
# 자세한 사용 가이드는 roboflow.com을 참조 하시기 바랍니다.

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="******************")
project = rf.workspace("user name").project("project name")
dataset = project.version(6).download("data format",location = "저장 경로")

In [ ]:
# 해당 프로젝트는 google colab 환경에서 수행하였습니다. 
# yolov5를 다운로드
# 아래 모든 과정은 해당 yolov5 github를 참조
%cd /content/drive/"본인로컬"
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
# 실행에 필요한 라이브러리 설치
%cd /content/drive/MyDrive/project2/yolov5
!pip install -r requirements.txt

In [ ]:
# custom 데이터셋을 저장
# 이미지의 경로를 나타내는 정보를 Yaml파일에 업데이트하는 과정

from glob import glob
 
# 이미지들의 주소 리스트로 만들어줌
train_img_list = glob('/content/drive/MyDrive/project2/yolov5/data0428_inc/train/images/*.jpg')
valid_img_list = glob('/content/drive/MyDrive/project2/yolov5/data0428_inc/valid/images/*.jpg')
test_img_list = glob('/content/drive/MyDrive/project2/yolov5/data0428_inc/test/images/*.jpg')
 
# 리스트를 txt파일로 생성
with open('/content/drive/MyDrive/project2/yolov5/data0428_inc/train.txt', 'w') as f:
   f.write('\n'.join(train_img_list) + '\n')
    
with open('/content/drive/MyDrive/project2/yolov5/data0428_inc/valid.txt', 'w') as f:
   f.write('\n'.join(valid_img_list) + '\n')

with open('/content/drive/MyDrive/project2/yolov5/data0428_inc/test.txt', 'w') as f:
   f.write('\n'.join(test_img_list) + '\n')   

In [ ]:
import yaml
 
with open('/content/drive/MyDrive/project2/yolov5/data0428_inc/data.yaml', 'r') as f:
  data = yaml.full_load(f)
    
#print(data)
 
data['train'] = '/content/drive/MyDrive/project2/yolov5/data0428_inc/train.txt'
data['val'] = '/content/drive/MyDrive/project2/yolov5/data0428_inc/valid.txt'
data['test'] = '/content/drive/MyDrive/project2/yolov5/data0428_inc/test.txt'
 
with open('/content/drive/MyDrive/project2/yolov5/data0428_inc/data.yaml', 'w') as f:
   yaml.dump(data, f)

In [ ]:
# yolov5는 wandb와의 연동을 지원합니다.
# wandb를 통해 시각화 및 학습과정을 확인가능
!pip install wandb

In [ ]:
# 처음 이용한다면 출력되는 url접속을 통해 회원가입 후 key를 입력한다.
!wandb login

In [ ]:
!wandb init

In [ ]:
# 학습 진행전 gpu캐시를 초기화
import torch
torch.cuda.empty_cache()

In [ ]:
%cd /content/drive/MyDrive/project2/yolov5
# imgz : 학습에 적용될 이미지 사이즈 (원본 이미지와 동일한 사이즈로 진행하는 것이 학습 효율이 좋다고 함)
# custom 데이터셋을 사전 학습된 모델로 학습 (가중치를 업데이트 하며 학습이 진행되고, 최적의 가중치를 도출)
!python train.py --imgsz 1024 --batch-size 10 --epochs 200 --data /content/drive/MyDrive/project2/yolov5/data0428_inc/data.yaml --cfg ./models/yolov5l.yaml --weights yolov5l.pt --name 0428_inc_L_1024

In [ ]:
# val.py를 실행하여 학습된 가중치로 검증
!python val.py --imgz 1024 --data /content/drive/MyDrive/project2/yolov5/data0428/data.yaml --weights /content/drive/MyDrive/project2/yolov5/runs/train/0428_inc_L_1024/weights/best.pt

In [ ]:
# 학습 정보를 통해 test셋 이미지를 예측 및 검출
# 단일 파일과 폴더째 둘다 가능
# runs/detect 폴더에 결과 이미지가 저장
!python detect.py --imgsz 1024 --weights /content/drive/MyDrive/project2/yolov5/runs/train/0428_inc_L_1024/weights/best.pt --source /content/drive/MyDrive/project2/yolov5/data/images/pattern_1-1-1-36-D_jpg.rf.0d292b60b819dfd97b8bacc10d72e8d7.jpg --save-crop